# simple autoEncoder

In [1]:
import os

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

In [2]:
if not os.path.exists('mlp_img/'):
    os.mkdir('mlp_img/')

In [3]:
def to_img(x):
    x = 0.5*(x+1)
    x = x.clamp(0,1)
    x = x.view(x.size(0),1,28,28)
    return x

In [4]:
num_epochs = 100
batch_size = 32
learning_rate = 1e-3

In [5]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [6]:
dataset = MNIST('data/',transform=img_transform)
dataloader = DataLoader(dataset,batch_size=batch_size,shuffle=True)

In [10]:
class autoencoder(nn.Module):
    def __init__(self,):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28*28,128),
            nn.ReLU(True),
            nn.Linear(128,64),
            nn.ReLU(True),
            nn.Linear(64,12),
            nn.ReLU(True),
            nn.Linear(12,3)
        )
        self.decoder = nn.Sequential(
            nn.Linear(3,12),
            nn.ReLU(True),
            nn.Linear(12,64),
            nn.ReLU(True),
            nn.Linear(64,128),
            nn.ReLU(True),
            nn.Linear(128,28*28),
            nn.Tanh()
        )
    def forward(self,x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [11]:
model = autoencoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate,weight_decay=1e-5)

In [12]:
for epoch in range(num_epochs):
    for data in dataloader:
        img,_ = data
        img = img.view(img.size(0),-1)
        img = Variable(img)
        output = model(img)
        loss = criterion(output,img)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1,num_epochs,loss.data[0]))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic,'mlp_img/img_{}.png'.format(epoch))

epoch [1/100], loss:0.1727
epoch [2/100], loss:0.1518
epoch [3/100], loss:0.1585
epoch [4/100], loss:0.1366
epoch [5/100], loss:0.1425
epoch [6/100], loss:0.1252
epoch [7/100], loss:0.1253
epoch [8/100], loss:0.1298
epoch [9/100], loss:0.1495
epoch [10/100], loss:0.1156
epoch [11/100], loss:0.1486
epoch [12/100], loss:0.1227
epoch [13/100], loss:0.1322
epoch [14/100], loss:0.1257
epoch [15/100], loss:0.1626
epoch [16/100], loss:0.1490
epoch [17/100], loss:0.1164
epoch [18/100], loss:0.1157
epoch [19/100], loss:0.1260
epoch [20/100], loss:0.1227
epoch [21/100], loss:0.1144
epoch [22/100], loss:0.1368
epoch [23/100], loss:0.1069
epoch [24/100], loss:0.1113
epoch [25/100], loss:0.1392
epoch [26/100], loss:0.1169
epoch [27/100], loss:0.1026
epoch [28/100], loss:0.1272
epoch [29/100], loss:0.1072
epoch [30/100], loss:0.1369
epoch [31/100], loss:0.1161
epoch [32/100], loss:0.1039
epoch [33/100], loss:0.1105
epoch [34/100], loss:0.1314
epoch [35/100], loss:0.1154
epoch [36/100], loss:0.1451
e

In [14]:
torch.save(model.state_dict(),'sim_autoencoder.pth')